## Algoritmo Genético

Los **algoritmos genéticos** (**GA**, **genetic algorithm**) son métodos de optimización inspirados en los principios de la evolución natural. Imitan el proceso de selección natural donde los individuos más aptos tienen mayor probabilidad de sobrevivir y reproducirse. 

Estos algoritmos son especialmente útiles para:
- Problemas con múltiples óptimos locales
- Espacios de búsqueda complejos y no lineales
- Optimización cuando no existen métodos analíticos directos

## Componentes Básios
- *Población*: Conjunto de soluciones candidatas (individuos)
- *Función de Aptitud*: Evalúa la calidad de cada solución
- *Selección*: Escoge los mejores individuos para reproducción
- *Cruzamiento*: Combina características de padres para crear hijos
- *Mutación*: Introduce cambios aleatorios para mantener diversidad
- *Reemplazo*: Genera una nueva población para la siguiente generación

## Flujo básico
1. Inicializar aptitud de cada individuo
2. Evaluar aptitud de cada individuo
3. Seleccionar padres basados en su aptitud 
4. Aplicar operadores genéticos (cruzamiento y mutación)
5. Crear nueva generación
6. Repetir pasos 2-5 hasta cumplir *criterio de parada*


## Ejemplo: Optimización de Funciones Matemáticas 
Uso del algoritmo genético para minimizar (o maximizar) funciones matemáticas. Primero se va a mostrar una implementación para optimizar funciones de única variable.

In [ ]:
# configuración inicial
import math 
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Literal, Tuple, Callable

class GeneticOptimizer:
  "Optimizador basado en algoritmos genéticos para minimizar (o maximizar) funciones objetivo"
  def __init__(self, 
      objetive_function: Callable[[float], float],
      bounds: Tuple[float, float],
      optimization_type: Literal["minimize", "maximize"] = "minimize",
      population_size: int = 50,
      mutation_rate: float = 0.1,
      crossover_rate: float = 0.8,
      elite_size: int = 2
    ) -> None:
    """Inicializa el optimizador genético.

    Args:
      objetive_function (Callable[[float], float]): función matemática
      bounds (Tuple[float, float]): límites del dominio de búsqueda (min, max)
      optimization_type (Literal[&quot;minimize&quot;, &quot;maximize&quot;], optional): tipo de optimización. Defaults to "minimize".
      population_size (int, optional): tamaño de la población. Defaults to 50.
      mutation_rate (float, optional): tasa de mutación. Defaults to 0.1.
      crossover_rate (float, optional): tasa de cruzamiento. Defaults to 0.8.
      elite_size (int, optional): número de individuos élite a conservar. Defaults to 2.
    """
    self.objective_function: Callable[[float], float] = objetive_function
    self.bounds: Tuple[float, float] = bounds
    self.optimization_type: Literal["minimize", "maximize"] = optimization_type
    self.population_size: int = population_size
    self.mutation_rate: float = mutation_rate
    self.crossover_rate: float = crossover_rate
    self.elite_size: int = elite_size
    
    self.population: List[float] = []
    self.fitness_history: List[float] = []
    self.best_individual_history: List[float] = []
    
  def _initialize_population(self) -> None:
    "Genera una población inicial aleatoria dentro de los límites del dominio de búsqueda"
    self.population = [
      random.uniform(self.bounds[0], self.bounds[1]) for _ in range(self.population_size)
    ]
  
  def _calculate_fitness(self, individual: float) -> float:
    """Calcula la aptitud de un individuo según el tipo de optimización.

    Args:
      individual (float): valor del individuo a evaluar

    Returns:
      float: valor de aptitud del individuo
    """
    objetive_value: float = self.objective_function(individual)
    if self.optimization_type == "minimize":
      # Para minimización: convertir a maximización usando inverso
      return 1.0 / (1.0 + abs(objetive_value))  
    else:
      # Para maximización: usar el valor obtenido directamente
      return objetive_value  
  
  def _selection(self) -> List[float]:
    """Selecciona individuos para la siguiente generación usando selección por torneo.

    Returns:
      List[float]: lista de individuos seleccionados
    """
    selected: List[float] = []
    
    # conservar élite basado en el objetivo real
    population_with_objetive: List[Tuple[float, float]] = [
      (individual, self._calculate_fitness(individual)) for individual in self.population
    ]
    
    # ordenar según tipo de optimización
    if self.optimization_type == "minimize":
      population_with_objetive.sort(key=lambda x: x[1])
    else: # maximize
      population_with_objetive.sort(key=lambda x: x[1], reverse=True)
    
    # agregar individuos élite
    for i in range(self.elite_size):
      selected.append(population_with_objetive[i][0])
    
    # selección por torneo para el resto (usando fitness para selección)
    tournament_size: int = 3
    while len(selected) < self.population_size:
      tournament: List[float] = random.sample(self.population, tournament_size)
      winner: float = max(tournament, key=self._calculate_fitness)
      selected.append(winner)
      
    return selected
  
  def _crossover(self, parent1:float, parent2:float) -> Tuple[float, float]:
    """Realiza cruzamiento entre dos padres usando cruzamiento aritmético

    Args:
      parent1 (float): primer padre
      parent2 (float): segundo padre

    Returns:
      Tuple[float, float]: tupla con los descendientes generados
    """
    if random.random() < self.crossover_rate:
      # cruzamiento aritmético
      alpha: float = random.random()
      child1 = alpha*parent1 + (1-alpha)*parent2
      child2 = (1-alpha)*parent1 + alpha*parent2
      
      # asegurar que los hijos estén dentro los límites
      min_bound,max_bound = self.bounds
      child1 = max(min_bound, min(max_bound, child1))
      child1 = max(min_bound, min(max_bound, child2))
      
      parent1, parent2 = child1, child2 
    return parent1, parent2
  
  def _mutate(self, individual: float) -> float:
    """Aplica mutacion gaussiana a un individuo.

    Args:
      individual (float): individuo a mutar

    Returns:
      float: individuo mutado
    """
    if random.random() < self.mutation_rate:
      mutation: float = random.gauss(0, 1)  # mutación gaussiana
      individual += mutation
      
      # asegurar que el individuo esté dentro de los límites
      min_bound, max_bound = self.bounds
      individual = max(min_bound, min(max_bound, individual))
    return individual
  
  def optimize(self, generations: int = 100, verbose: bool = True) -> Tuple[float, float]:
    """Encuentra el óptimo usando el algoritmo genético

    Args:
      generations (int, optional): número de generaciones a ejecutar. Defaults to 100.
      verbose (bool, optional): si mostrar progreso durante la optimización. Defaults to True.

    Returns:
      Tuple[float, float]: tupla con el mejor individuo encontrado y su valor objetivo
    """
    self._initialize_population()
    self.fitness_history = []
    self.best_individual_history = []
    
    for generation in range(generations):
      # selección
      selected: List[float] = self._selection()
      
      # crear nueva población
      new_population: List[float] = []
      
      # procesar pares para cruzamiento
      for i in range(0, len(selected), 2):
        if i + 1 < len(selected):
          parent1, parent2 = selected[i], selected[i + 1]
          child1, child2 = self._crossover(parent1, parent2)
          child1 = self._mutate(child1)
          child2 = self._mutate(child2)
          new_population.extend([child1, child2])
        else:
          # si hay un número impar de seleccionados, agregar el último sin cruzamiento
          child = self._mutate(selected[i])
          new_population.append(child)
        
      # asegurar tamaño de población
      self.population = new_population[:self.population_size]
      
      # encontrar mejor individuo de esta generación 
      population_with_objetive: List[Tuple[float, float]] = [
        (individual, self.objective_function(individual)) 
        for individual in self.population
      ]
      
      if self.optimization_type == "minimize":
        best_individual, best_objective = min(population_with_objetive, key=lambda x: x[1])
      else: # maximize
        best_individual, best_objective = max(population_with_objetive, key=lambda x: x[1])
      
      # registrar generación
      self.fitness_history.append(best_objective)
      self.best_individual_history.append(best_individual)
      
      # mostrar información si verbose es True 
      if verbose and generation % 10 == 0:
        print(f"Generación {generation}: Mejor individuo (x) = {best_individual}, f(x) = {best_objective} ({self.optimization_type})")
    
    # encontrar el mejor individuo final
    population_with_objetive: List[Tuple[float, float]] = [
      (individual, self.objective_function(individual)) 
      for individual in self.population
    ]
    
    if self.optimization_type == "minimize":
      best_individual, best_objective = min(population_with_objetive, key=lambda x: x[1])
    else:
      best_individual, best_objective = max(population_with_objetive, key=lambda x: x[1])

    return best_individual, best_objective
  
  def plot_convergence(self) -> None:
    "Muestra la gráfica de convergencia de la optimización"
    plt.figure(figsize=(18, 5))

    # Gráfica 1: Convergencia del valor objetivo
    plt.subplot(1, 3, 1)
    plt.plot(self.fitness_history, 'b-', linewidth=2)
    plt.title(f'Convergencia del Algoritmo Genético ({self.optimization_type})')
    plt.xlabel('Generación')
    plt.ylabel('Mejor Valor Objetivo')
    plt.grid(True, alpha=0.3)
    
    # Gráfica 2: Evolución del mejor individuo
    plt.subplot(1, 3, 2)
    plt.plot(self.best_individual_history, 'r-', linewidth=2)
    plt.title('Evolución del Mejor Individuo (x)')
    plt.xlabel('Generación')
    plt.ylabel('Valor de x')
    plt.grid(True, alpha=0.3)
    
    # Gráfica 3: Función objetivo
    plt.subplot(1, 3, 3)
    x_vals = np.linspace(self.bounds[0], self.bounds[1], 100)
    y_vals = np.array([self.objective_function(x) for x in x_vals])
    plt.plot(x_vals, y_vals, 'g-', linewidth=2)
    # graficar historial de mejores individuos
    history_x = np.array(self.best_individual_history)
    history_y = np.array([self.objective_function(x) for x in history_x]) 
    plt.scatter(history_x, history_y, color='orange', label='Mejores Individuos', s=50)
    # marcar el mejor individuo final 
    best_idx = np.argmin(self.fitness_history) if self.optimization_type == "minimize" else np.argmax(self.fitness_history)
    best_x = history_x[best_idx]
    best_y = history_y[best_idx]
    plt.scatter(best_x, best_y, color='red', label='Mejor Individuo Final', s=100, edgecolor='black')
    plt.title("Historial sobre la Función Objetivo")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_function(objective_function: Callable[[float], float], bounds: Tuple[float, float]) -> None:
  """Grafica la función objetivo en el rango especificado.

  Args:
    objective_function (Callable[[float], float]): función matemática a graficar
    bounds (Tuple[float, float]): límites del dominio de búsqueda (min, max)
  """
  x_vals = np.linspace(bounds[0], bounds[1], 100)
  y_vals = np.array([objective_function(x) for x in x_vals])
  
  plt.figure(figsize=(10, 5))
  plt.plot(x_vals, y_vals, 'b-', linewidth=2)
  plt.title("Función Objetivo")
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.grid(True, alpha=0.3)
  plt.show()

In [ ]:
optimization_type: Literal["minimize", "maximize"] = "minimize"
def objective_function_example(x: float) -> float:
  "Función con mínimo: f(x) = x^2 + x + 1"
  return x**2 + x + 1

plot_function(objective_function_example, (-10, 10))

In [ ]:
optimization_type: Literal["minimize", "maximize"] = "maximize"
def objective_function_example(x: float) -> float:
  "Función con máximo: f(x) = -(x^2 + x + 1)"
  return -(x**2 + x + 1)

plot_function(objective_function_example, (-10, 10))

In [ ]:
optimization_type: Literal["minimize", "maximize"] = "maximize"
def objective_function_example(x: float) -> float:
  "Función con máximo: f(x) = -(x^2 + x + 1)"
  return (math.cos(x) // (x**2 + x + 1)) + math.sin(x)

plot_function(objective_function_example, (-100, 100))

In [ ]:
print("Optimización con Algoritmo Genético")
print("===================================")

optimizer = GeneticOptimizer(
  objetive_function=objective_function_example,
  bounds=(-100.0, 100.0),
  optimization_type=optimization_type,
  population_size=50,
  mutation_rate=0.2,
  crossover_rate=0.8,
  elite_size=20
)

best_x, best_value = optimizer.optimize(generations=1000, verbose=True)
print(f"Mejor x encontrado:   {best_x}")
print(f"Valor objetivo:       {best_value} ({optimization_type})")

optimizer.plot_convergence()